In [1]:
import numpy as np
import findspark
from functions.required_functions import (
    readFile,
    normalize,
    transform,
    get_block_data,
    train,
    accuracy,
)
from pyspark import SparkContext
import time
import json
import matplotlib.pyplot as plt
findspark.init()

In [2]:

# ==============================
# EXPERIMENT ORCHESTRATOR
# =============================
def run_experiment(partitions=None,nodes=None,file_path="data/botnet_reduced_10k_l.csv",nIter=20,learning_rate=1.5,lambda_reg=0,):
    results = []
    # Experimento variando partitions
    if partitions is not None:
        for n_partitions in partitions:
            result = _configurate_experiment(file_path,nIter,learning_rate,lambda_reg,n_partitions=n_partitions,)
            result["partitions"] = n_partitions
            results.append(result)
        # Guardar resultados en JSON
        with open("results/experiment4.json", "w") as f:
            json.dump(results, f, indent=4)
        _plot_results(
            results,
            x_key="partitions",
            title="Execution Time vs Partitions",
            xlabel="Number of partitions",
            filename="results/execution_time_vs_partitions.png",
        )
        _plot_stacked_times(
            results,
            x_key="partitions",
            filename="results/stacked_times_vs_partitions.png",
        )
    # Experimento variando workers
    elif nodes is not None:
        for n_node in nodes:
            result = _configurate_experiment(file_path,nIter,learning_rate,lambda_reg,n_nodes=n_node,)
            result["nodes"] = n_node
            results.append(result)
        # Guardar resultados en JSON
        with open("results/experiment4.json", "w") as f:
            json.dump(results, f, indent=4)

        _plot_results(
            results,
            x_key="nodes",
            title="Execution Time vs Workers",
            xlabel="Number of workers",
            filename="results/execution_time_vs_workers.png",
        )

        _plot_stacked_times(
            results,
            x_key="nodes",
            filename="results/stacked_times_vs_workers.png",
        )
    return results


# ==============================
# SPARK CONTEXT LIFECYCLE
# ==============================
def _configurate_experiment(file_path,nIter,learning_rate,lambda_reg,n_partitions=None,n_nodes="*",):
    sc = SparkContext(
        master=f"local[{n_nodes}]",
        appName="Experiment_4",
    )
    try:
        result = _run_training(file_path,nIter,learning_rate,lambda_reg,sc,n_partitions,)
    finally:
        sc.stop()
    return result
# ==============================
# TRAINING PIPELINE
# ==============================


def _run_training(
    file_path,
    nIter,
    learning_rate,
    lambda_reg,
    sc,
    n_partitions,
):
    # READ
    start = time.time()
    data = readFile(file_path, sc)
    read_time = time.time() - start
    # REPARTITION
    repartition_time = 0
    if n_partitions is not None:
        start = time.time()
        data = data.repartition(n_partitions).cache()
        # Fuerza shuffle
        data.count()
        repartition_time = time.time() - start

    # NORMALIZE
    start = time.time()
    data = normalize(data).cache()
    data.count()
    normalize_time = time.time() - start
    # TRAIN
    start = time.time()
    w, b = train(
        data,
        nIter,
        learning_rate,
        lambda_reg,
    )
    train_time = time.time() - start
    # ACCURACY
    start = time.time()
    acc = accuracy(w, b, data)
    acc_calc_time = time.time() - start
    print(f"Accuracy: {acc:.4f}")
    total_time = (
        read_time + repartition_time + normalize_time + train_time + acc_calc_time
    )
    print(f"Tiempo total experimento: {total_time:.4f}s")

    return {
        "read_time": read_time,
        "repartition_time": repartition_time,
        "normalize_time": normalize_time,
        "train_time": train_time,
        "acc_calc_time": acc_calc_time,
        "total_time": total_time,
    }

# ==============================
# PLOTTING
# ==============================
def _plot_results(results, x_key, title, xlabel, filename):
    x = [r[x_key] for r in results]
    y = [r["total_time"] for r in results]

    plt.figure()
    plt.plot(x, y, marker="o")
    plt.xlabel(xlabel)
    plt.ylabel("Execution Time (s)")
    plt.title(title)
    plt.savefig(filename)
    plt.close()


def _plot_stacked_times(results, x_key, filename):
    x_labels = [r[x_key] for r in results]

    steps = ["read_time", "repartition_time", "normalize_time", "train_time", "acc_calc_time"]
    colors = ["#FF9999", "#66B2FF", "#99FF99", "#FFCC66", "#C266FF"]

    bottoms = [0]*len(results)
    plt.figure(figsize=(10, 6))

    for step, color in zip(steps, colors):
        heights = [r[step] for r in results]
        plt.bar(x_labels, heights, bottom=bottoms, color=color, label=step)
        bottoms = [b + h for b, h in zip(bottoms, heights)]

    plt.xlabel(x_key)
    plt.ylabel("Time (s)")
    plt.title("Time per step (stacked)")
    plt.legend()
    plt.savefig(filename)
    plt.close()

In [3]:
run_experiment(file_path="data/botnet_tot_syn_l.csv", partitions=list(range(1, 12*4)))


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/22 00:48:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/22 00:48:52 WARN BlockManager: Task 13 already completed, not releasing lock for rdd_10_0


Iteration 0/20, Cost: 0.5764, Accuracy: 0.6841


Iteration 2/20, Cost: 0.3170, Accuracy: 0.8905


Iteration 4/20, Cost: 0.2579, Accuracy: 0.9153


Iteration 6/20, Cost: 0.2317, Accuracy: 0.9238


...

Iteration 14/20, Cost: 0.1931, Accuracy: 0.9332


Iteration 16/20, Cost: 0.1890, Accuracy: 0.9340


Iteration 18/20, Cost: 0.1858, Accuracy: 0.9346


Accuracy: 0.9347
Tiempo total experimento: 149.2508s


[{'read_time': 0.38809919357299805,
  'repartition_time': 4.281903505325317,
  'normalize_time': 12.841432332992554,
  'train_time': 515.991021156311,
  'acc_calc_time': 8.067081928253174,
  'total_time': 541.5695381164551,
  'partitions': 1},
 {'read_time': 0.023748159408569336,
  'repartition_time': 2.6528072357177734,
  'normalize_time': 6.787390470504761,
  'train_time': 279.44835019111633,
  'acc_calc_time': 4.341438055038452,
  'total_time': 293.2537341117859,
  'partitions': 2},
 {'read_time': 0.016373395919799805,
  'repartition_time': 2.6048569679260254,
  'normalize_time': 4.940197706222534,
  'train_time': 195.72052526474,
  'acc_calc_time': 3.0726113319396973,
  'total_time': 206.35456466674805,
  'partitions': 3},
 {'read_time': 0.011455774307250977,
  'repartition_time': 2.518641471862793,
  'normalize_time': 4.001041889190674,
  'train_time': 157.27515149116516,
  'acc_calc_time': 2.4726595878601074,
  'total_time': 166.278950214386,
  'partitions': 4},
 {'read_time': 0.